In [ ]:
import requests
import tqdm
import os
import json

In [ ]:
API_SECRET = ""
API_USER = ""
FILES_FOLDER = ""

In [ ]:
## Ocr a doc
import os
from hrflow import Hrflow

VALID_EXTENSIONS = [
    ".pdf",
    ".png",
    ".jpg",
    ".jpeg",
    ".bmp",
    ".doc",
    ".docx",
    ".rtf",
    ".dotx",
    ".odt",
    ".odp",
    ".ppt",
    ".pptx",
    ".rtf",
    ".msg",
]
INVALID_FILENAME = [".", ".."]
def is_valid_extension(file_path):
    ext = os.path.splitext(file_path)[1]
    if not ext:
        return False
    return ext in VALID_EXTENSIONS or ext.lower() in VALID_EXTENSIONS


def is_valid_filename(file_path):
    name = os.path.basename(file_path)
    return name not in INVALID_FILENAME

def get_files_from_dir(dir_path, is_recurcive):
    file_res = []
    files_path = os.listdir(dir_path)

    for file_path in files_path:
        true_path = os.path.join(dir_path, file_path)
        if os.path.isdir(true_path) and is_recurcive:
            if is_valid_filename(true_path):
                file_res += get_files_from_dir(true_path, is_recurcive)
            continue
        if is_valid_extension(true_path):
            file_res.append(true_path)
    return file_res

In [ ]:
files_list = get_files_from_dir(FILES_FOLDER,True)
files_list

In [ ]:
def tagger(api_secret,api_user,text,labels,context=""):
    url = "https://api.hrflow.ai/v1/text/tagging"

    payload = {
        "algorithm_key": "tagger-hrflow-dynamic",
        "texts": [text],
        "dynamic_labels": labels,
        "top_n": 1,
        "dynamic_context": context
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "X-API-KEY": api_secret,
        "X-USER-EMAIL": api_user,
    }

    response = requests.post(url, json=payload, headers=headers)
    if response.status_code != 200:
        print(f"HTTP error: {response.status_code}")
        return None

    response_data = response.json()
    if (len(response_data["data"]) > 0 and isinstance(response_data["data"][0], dict) and 
    "tags" in response_data["data"][0] and isinstance(response_data["data"][0]["tags"], list) and 
    len(response_data["data"][0]["tags"]) > 0):
        tag = response_data["data"][0]["tags"][0]
    else:
        tag = None  
    return tag

In [ ]:
from tqdm import tqdm
import time
failed = []
labels = ["resume", "coverletter", "references", "diploma", "certificate", "permit", "license", "passport"]
headers = {
  'X-USER-EMAIL': API_USER,
  'X-API-KEY': API_SECRET
}
url = "https://api.hrflow.ai/v1/text/ocr"

for file_path in tqdm(files_list):
    filename = os.path.basename(file_path)
    try:
        with open(file_path, "rb") as file:
            payload = {}
            files=[('file',(filename,file,'application/pdf'))]
            response = requests.request("POST", url, headers=headers, data=payload, files=files)
        text = response.json()["data"]["text"]
        tag = tagger(API_SECRET,API_USER,text,labels)
        print(f"File : {file_path} -> Tag : {tag if tag else 'No categorization'}")
    except Exception as e:
        failed.append(file_path)

In [ ]:
if failed:
    print(f"\nFailed files ({len(failed)}) :")
    for f in failed:
        print(f" - {f}")